# IE582- Homework 4

## Ahmet Buğra Taksuk - 2017402168


- In this homework, we are asked to come up with two representation to deal with Multiple Instance Learning problem. After finding two representation, we are also asked to apply these into two different models and get Cross Validation score on training data. For the first represetation, I created two arrays which is basically mean of the bag +- the standard deviation of the bag. Hence, our feature vector represented the sample statistics of the bag. For the second model, I found the most further points in the bag and concatanated them to use as feature vector. The related dataset can be found on https://archive.ics.uci.edu/ml/datasets/Musk+(Version+1) . 

In [57]:
# Necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from scipy.spatial import distance
sns.set(rc={'figure.figsize':(8,6)})

In [328]:
data = pd.read_csv("Musk1.csv", header = None)
data.rename(columns = {0:'Bag_Class', 1:'Bag_ID'}, inplace = True)
data

,Bag_Class,Bag_ID,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167
0,1,1,42,-198,-109,-75,-117,11,23,-88,-28,-27,-232,-212,-66,-286,-287,-300,-57,-75,-192,-184,-66,-18,-50,111,110,18,-18,-127,25,63,-117,-114,-47,9,-135,26,-175,73,-143,71,-177,-85,-30,-282,-280,-249,-135,-11,-139,-105,-142,-32,-9,-48,147,1,40,-170,35,33,-101,-195,26,-5,-144,48,-165,18,-133,15,-146,-148,-146,-246,-216,-181,-37,-212,-216,-174,-20,8,-120,-38,-7,11,-156,-39,-7,82,-202,-15,-115,-46,26,-49,-166,32,-141,76,-206,26,-257,-289,-304,-163,-117,-17,-247,-283,-244,-64,-35,-32,-10,57,110,25,6,-117,80,149,130,-110,-134,-14,35,51,11,-187,13,-138,-67,-163,-201,-19,45,-115,-11,-37,-100,77,78,60,-178,-102,-118,-33,-104,41,-77,-120,-111,-168,-54,-195,-238,-74,-129,-120,-38,30,48,-37,6,30
1,1,1,42,-191,-142,-65,-117,55,49,-170,-45,5,-325,-115,-107,-281,-257,-303,54,-154,-101,-47,-31,-28,1,191,72,-38,50,-64,-63,98,-117,-113,-46,2,-135,25,-159,1,-143,38,-169,-85,-31,-323,-234,-334,-88,-73,-109,-4,-75,-31,-14,-137,105,-94,90,-132,3,7,-101,-195,26,109,-130,48,-165,-80,-133,-50,-153,-148,-297,-194,-96,-181,-5,-289,-107,-179,-21,4,-34,115,15,15,-74,-164,-73,131,-202,-15,-115,-308,26,-50,-33,25,-154,75,-191,26,-227,-309,-284,-266,-163,-122,-185,-234,-212,0,-3,-3,22,-12,156,36,82,31,82,70,111,-110,-133,-13,-26,75,-107,-187,13,-138,-77,-129,-224,-89,51,-70,-19,-35,-29,3,43,10,-178,-102,-119,-57,-70,53,-77,-123,-111,-168,-54,-195,-238,-302,60,-120,-39,31,48,-37,5,30
2,1,1,42,-191,-142,-75,-117,11,49,-161,-45,-28,-278,-115,-67,-274,-285,-303,53,-154,-100,-183,-31,-28,1,110,110,-38,51,-64,25,63,-117,-113,-47,10,-135,26,-175,2,-143,38,-168,-85,-31,-293,-246,-326,-89,-73,-108,-105,-75,-31,-14,-117,148,-93,90,-132,35,33,-101,-195,26,-5,-144,49,-165,-80,-133,-50,-153,-148,-148,-194,-217,-181,-5,-278,-107,-163,-21,4,-34,-37,-8,15,-74,-164,-7,81,-202,-15,-115,-46,26,-49,-166,25,-154,76,-191,26,-254,-280,-291,-266,-164,-122,-247,-250,-233,0,-4,-3,-9,57,110,37,82,31,79,148,130,-110,-133,-13,35,51,11,-187,13,-138,-77,-129,-221,-89,52,-71,-19,-35,-29,3,43,10,-178,-102,-118,-57,-70,54,-77,-120,-111,-168,-54,-195,-238,-73,-127,-120,-38,30,48,-37,5,31
3,1,1,42,-198,-110,-65,-117,55,23,-95,-28,5,-301,-212,-107,-280,-284,-301,-56,-74,-192,-46,-66,-19,-50,191,73,18,-19,-128,-63,98,-117,-113,-46,3,-135,25,-159,73,-143,70,-177,-86,-31,-286,-280,-330,-135,-11,-138,-4,-143,-32,-9,-136,105,1,40,-170,2,6,-101,-195,26,109,-130,48,-165,18,-133,15,-146,-148,-263,-246,-96,-181,-37,-230,-216,-162,-20,8,-120,116,14,10,-156,-40,-72,131,-202,-15,-115,-308,26,-50,-34,32,-141,75,-206,26,-227,-282,-300,-183,-116,-16,-206,-283,-244,-65,-35,-32,22,-11,157,24,6,-117,83,69,111,-110,-133,-13,-27,76,-108,-187,13,-138,-67,-163,-201,-19,45,-115,-10,-37,-100,76,78,59,-178,-102,-118,-33,-104,41,-77,-123,-111,-168,-54,-195,-238,-302,60,-120,-39,30,48,-37,6,30
4,1,2,42,-198,-102,-75,-117,10,24,-87,-28,-28,-233,-212,-67,-286,-286,-299,-57,-74,-191,-182,-66,-18,-50,109,111,18,-18,-128,25,63,-117,55,-28,10,-131,66,-175,73,-128,71,-177,102,-31,-283,-280,-249,-135,-11,-137,-105,-143,-32,-9,-48,148,1,41,-170,34,33,-101,-156,24,-5,-144,58,-165,17,-123,15,-140,-113,-148,-246,-217,-181,-37,-212,-215,-173,-20,8,-120,-38,-8,10,-156,-40,-7,81,-202,70,-114,-46,28,-34,-167,32,-126,76,-206,134,-257,-289,-304,-162,-116,-16,-247,-282,-243,-64,-35,-32,-9,57,110,24,6,-117,80,148,130,-107,-108,-9,36,51,12,-187,17,-138,-67,-163,-201,-20,45,-116,-10,-37,-99,76,79,60,-177,-102,-118,-33,-104,41,-66,-120,-111,-120,97,-121,-238,-73,-127,51,128,144,

## 1 ) First Representation and related models
- In this representation, I am going to take consideration of sample statistics into account. For every bag I will represent it by the mean values for each feature -+ the standard deviation of the each feature for every instance in the bag. By doing so, if an instance is labeled as 1 whereas the others labeled as 0, the standard deviation will be higher and vice versa

In [419]:
from sklearn.metrics.pairwise import euclidean_distances
new_data = []
labels = []
for i in range(1,93):
    X = data[data["Bag_ID"] == i].drop(columns = ["Bag_Class", "Bag_ID"])
    norm1 = np.array((np.mean(X) + np.std(X)))
    norm2 =  np.array((np.mean(X) - np.std(X)))
    new_data.append(np.concatenate([norm1, norm2]))
    labels.append(data[data["Bag_ID"] == i]["Bag_Class"].values[0])
    


### Logistic Regression 

In [383]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
y_train = np.array(labels)
X_train = np.array(new_data)
scores = []
# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=5 ,random_state=0)

# create model
model = LogisticRegression(max_iter = 10000) 
# evaluate model
np.mean(cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv))


0.8826666666666665

### KNeighborsClassifier

In [428]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
y_train = np.array(labels)
X_train = np.array(new_data)
scores = []
# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=5 ,random_state=0)

# create model
model = KNeighborsClassifier(n_neighbors=2, metric = 'euclidean')
# evaluate model
np.mean(cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv))

0.882888888888889

## 2 ) Second Representation and related models
- In this represenation, I am going to find the two most further point in bag and concatanete them as if they are a single point. So the representataion will be two extreme point of the bag which allows us to identify significant differences in bag such as one labeled as one and the other one labeled as zero. 

In [434]:
def find_two(bag):
    x,y = 0,0
    for i in range(len(bag)):
        max_dist = -100000000000
        for j in range(i+1,len(bag)):
            dist = distance.euclidean(bag[i],bag[j])
            if(dist>=max_dist):
                x , y = i,j 
                max_dist = dist
    return x,y

In [435]:
from sklearn.metrics.pairwise import euclidean_distances
new_data = []
labels = []
for i in range(1,93):
    X = data[data["Bag_ID"] == i].drop(columns = ["Bag_Class", "Bag_ID"]).values
    x,y = find_two(X)
    new_data.append(np.concatenate([X[x], X[y]]))
    labels.append(data[data["Bag_ID"] == i]["Bag_Class"].values[0])
    


### Logistic Regression 

In [436]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
y_train = np.array(labels)
X_train = np.array(new_data)
scores = []
# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=5 ,random_state=0)

# create model
model = LogisticRegression(max_iter = 10000) 
# evaluate model
np.mean(cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv))
        

0.8053333333333333

### KNeighborsClassifier

In [437]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
y_train = np.array(labels)
X_train = np.array(new_data)
scores = []
# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=5 ,random_state=0)

# create model
model = KNeighborsClassifier(n_neighbors=2, metric = 'euclidean') 
# evaluate model
np.mean(cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv))

0.7826666666666665